# Example receiving sales data from a Kafka stream, performing aggregation on sales by state
<font color='steelblue'>
<br>
<span style="font-family:Arial; font-size:1.6em;">
Sales data being streamed via Kafka, perform sales aggregation by state<br>
<br>
Steps to test the script: <br>
<ol>
    <li> Start the zookeeper, kafka brokers</li>
    <li> Create topic</li>
    <li> Run all the cells</li>
    <li> Run the producer code to generate the messages</li>
    <li> Observe the messages in the console (only 20 will be printed)</li>
</ol>
 </span>

</font>
<br>
<font color='gray'>
<span style="font-family:Arial; font-size:1.4em;">
    <b>Following are the steps in the processing:<br></b>
    <ul>
        <li><b>Read Stream:</b> Create kafka read stream</li>
        <li><b>Extract:</b> Extract the message from the stream</li>
        <li><b>Transform:</b> Trasnsform each row into a table (dataframe)</li>
        <li><b>Aggregate:</b> Aggregate the sales data by state</li>
        <li><b>Write Stream:</b> Write the stream to console</li>
    </ul>
</span>
</font>

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
# Want to write the print statements to console rather
# than output of jupyter cell - redirect stdout to connsole
import sys
sys.stdout = open(1, 'w')

In [ ]:
spark = SparkSession\
        .builder\
        .appName("State-Sales-Aggregation")\
        .getOrCreate()

In [ ]:
 spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# define the topic name
topicName = "sales-data"

# define where to start reading from
startFrom = "earliest"

## Define the read stream
<h3>
<ol>
    <li>Read the stream into dataframe</li>
    <li>Check the dataframe is streaming and print the schema</li>
</ol>
</h3>

In [ ]:
df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", topicName) \
      .option("startingOffsets", startFrom) \
      .load()

In [ ]:
# Check whether input data is streaming or not
print(" ")
print("Is the stream ready?")
print(df.isStreaming)

In [ ]:
# Print Schema
print(" ")
print("Schema of the input stream: ")
print(df.printSchema)

In [ ]:
# create a comma separated values for each row
salesDF = df.select(col("value").cast(StringType())).alias('csv').select('csv.*')

In [ ]:
# assign column names to the dataframe
sdf = salesDF.selectExpr("split(value,',')[0] as product" \
                 ,"split(value,',')[1] as city" \
                 ,"split(value,',')[2] as state" \
                 ,"split(value,',')[3] as country" \
                 ,"split(value,',')[4] as sales"
                 )

## Aggregate sales by state and put in new column "tot_sales"

In [ ]:
# Aggregate sales by states and then order them by the highest sales values
salesAgg = sdf.groupBy("state")\
                      .agg({"sales": "sum"})\
                      .withColumnRenamed("sum(sales)", "tot_sales")\
                      .orderBy("tot_sales", ascending=False)

<font color='tomato'>
    <span style="font-family:Arial; font-size:1.4em;">
        <i>If the output mode is <b>update</b> then the above code will give  error (sort is supported only on complete mode)<br> use the following code with update mode (comment the above code)</i>
    </span>
</font>

## Write the aggregate dataframe to console

In [ ]:
query = salesAgg.writeStream\
                  .outputMode('complete')\
                  .format('console')\
                  .option("truncate", "false")\
                  .start()

In [ ]:
query.awaitTermination()